
# Apache Cassandra Data Modeling and Query Execution
## Project Overview
This notebook focuses on creating data models and tables in Apache Cassandra to address the following queries:
1. Retrieve artist, song title, and song's length for a specific `sessionId` and `itemInSession`.
2. Retrieve artist, song (sorted by `itemInSession`), and user (first and last name) for a specific `userId` and `sessionId`.
3. Retrieve all users who listened to a specific song.

Each query has been carefully modeled to optimize data partitioning, retrieval, and performance while adhering to Apache Cassandra's design principles.

The notebook is structured as follows:
- **Data Modeling and Table Creation**
- **Data Insertion**
- **Query Execution and Results**
- **Clean-up and Conclusion**


# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/workspace/home


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [7]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [8]:
# TO-DO: Create a Keyspace 
# TO-DO: Create a Keyspace
session.execute("""
    CREATE KEYSPACE IF NOT EXISTS music_history
    WITH replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
""")


#### Set Keyspace

In [9]:
# TO-DO: Set KEYSPACE to the keyspace specified above
session.set_keyspace('music_history')


### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'





### Data Modeling Explanation
- **Table Name**: The table is named based on the query it serves. 
- **Partition Key**: Chosen to optimize data retrieval based on query parameters.
- **Clustering Column**: Used to maintain sort order or uniqueness within a partition.

**Note**: The combination of `PRIMARY KEY` (Partition Key + Clustering Column) ensures rows are uniquely identified and efficiently retrievable.


In [11]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below#
import csv

file = 'event_datafile_new.csv'

with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO session_songs (sessionId, itemInSession, artist, song, length)
            VALUES (%s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table


### Query Execution
This query retrieves the required data based on the defined partition key and clustering columns, without using `ALLOW FILTERING`.


In [ ]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
query = """
    SELECT artist, song, length 
    FROM session_songs 
    WHERE sessionId = 338 AND itemInSession = 4
"""
rows = session.execute(query)
for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, Length: {row.length}")

import pandas as pd
df = pd.DataFrame(rows)
print(df)


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS


### Data Modeling Explanation
- **Table Name**: The table is named based on the query it serves. 
- **Partition Key**: Chosen to optimize data retrieval based on query parameters.
- **Clustering Column**: Used to maintain sort order or uniqueness within a partition.

**Note**: The combination of `PRIMARY KEY` (Partition Key + Clustering Column) ensures rows are uniquely identified and efficiently retrievable.


In [16]:
# Insert data into user_sessions table
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO user_sessions (userId, sessionId, itemInSession, artist, song, firstName, lastName)
            VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))



### Query Execution
This query retrieves the required data based on the defined partition key and clustering columns, without using `ALLOW FILTERING`.


In [ ]:
# Verify the data for userId = 10 and sessionId = 182
query = """
    SELECT artist, song, firstName, lastName 
    FROM user_sessions 
    WHERE userId = 10 AND sessionId = 182
"""
rows = session.execute(query)
for row in rows:
    print(f"Artist: {row.artist}, Song: {row.song}, User: {row.firstname} {row.lastname}")

import pandas as pd
df = pd.DataFrame(rows)
print(df)



### Data Modeling Explanation
- **Table Name**: The table is named based on the query it serves. 
- **Partition Key**: Chosen to optimize data retrieval based on query parameters.
- **Clustering Column**: Used to maintain sort order or uniqueness within a partition.

**Note**: The combination of `PRIMARY KEY` (Partition Key + Clustering Column) ensures rows are uniquely identified and efficiently retrievable.


In [27]:
with open(file, encoding='utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)  # skip header
    
    for line in csvreader:
        query = """
            INSERT INTO song_listeners (song, userId, firstName, lastName)
            VALUES (%s, %s, %s, %s)
        """
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        


### Query Execution
This query retrieves the required data based on the defined partition key and clustering columns, without using `ALLOW FILTERING`.


In [ ]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query = """
    SELECT firstName, lastName 
    FROM song_listeners 
    WHERE song = 'All Hands Against His Own'
"""
rows = session.execute(query)
for row in rows:
    print(f"User: {row.firstname} {row.lastname}")
import pandas as pd
df = pd.DataFrame(rows)
print(df)


### Drop the tables before closing out the sessions

In [29]:
## TO-DO: Drop the table before closing out the sessions
session.execute("DROP TABLE IF EXISTS session_songs")
session.execute("DROP TABLE IF EXISTS user_sessions")
session.execute("DROP TABLE IF EXISTS song_listeners")
print("Tables dropped successfully.")


Tables dropped successfully.


### Close the session and cluster connection¶

In [30]:
# Close the session and cluster connection
session.shutdown()
cluster.shutdown()
print("Cassandra session and cluster connection closed.")


Cassandra session and cluster connection closed.



## Clean-up and Conclusion
The tables have been dropped, and the Cassandra session and cluster connection have been closed to ensure resource cleanup.

This notebook demonstrates how to correctly model data in Apache Cassandra to answer specific queries, optimize data partitioning, and avoid `ALLOW FILTERING`. The outputs have been presented in a clear and readable format using Pandas DataFrames.

**Best Practices Followed**:
- Logical table and column naming.
- Efficient use of partition keys and clustering columns.
- Avoidance of `ALLOW FILTERING`.
- Modular and clean notebook structure.
